In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv("train.csv")
score=pd.read_csv("score.csv")

In [3]:
train.drop('exact_item_bought',axis=1,inplace=True)
score.drop('exact_item_bought',axis=1,inplace=True)

In [4]:
train.head()

,id,chain,offer,market,repeater,offerdate,category,quantity,company,offervalue,brand,avg_purchaseamount,avg_purchasequantity,match
0,1048253332,214,1198271,8,f,2013/3/27,5558,1,107120272,1.50,5072,0.000000,0.000000,0
1,106372514,17,1197502,4,f,2013/3/28,3203,1,106414464,0.75,13474,0.000000,0.000000,0
2,1064972209,214,1208251,8,f,2013/4/23,2202,1,104460040,2.00,3718,0.000000,0.000000,0
3,1070619072,46,1208329,15,t,2013/4/23,2119,1,108079383,1.00,6926,4.517586,1.551724,0
4,1080684610,214,1197502,8,t,2013/4/19,3203,1,106414464,0.75,13474,2.690000,1.000000,1


In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
train["repeater"] = labelencoder_X.fit_transform(train['repeater'])

In [6]:
train.iloc[[~train.category.isin(score.category)],[6]]='unknown'
score.iloc[[~score.category.isin(train.category)],[5]]='unknown'

In [7]:
train.iloc[[~train.brand.isin(score.brand)],[10]]='unknown'
score.iloc[[~score.brand.isin(train.brand)],[9]]='unknown'

In [8]:
train.iloc[[~train.company.isin(score.company)],[8]]='unknown'
score.iloc[[~score.company.isin(train.company)],[7]]='unknown'

In [9]:
train.iloc[[~train.offer.isin(score.offer)],[2]]='unknown'
score.iloc[[~score.offer.isin(train.offer)],[2]]='unknown'

In [10]:
train.to_csv(r'C:\Users\flori\Documents\Python Scripts\datamining\out1.csv',index=False)
score.to_csv(r'C:\Users\flori\Documents\Python Scripts\datamining\out2.csv',index=False)

In [4]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import operator

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from C:\Users\flori\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\flori\AppData\Local\Temp\tmphteradps
  JVM stdout: C:\Users\flori\AppData\Local\Temp\tmphteradps\h2o_flori_started_from_python.out
  JVM stderr: C:\Users\flori\AppData\Local\Temp\tmphteradps\h2o_flori_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,1 month and 10 days
H2O cluster name:,H2O_from_python_flori_u7riuq
H2O cluster total nodes:,1
H2O cluster free memory:,1.965 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [6]:
path = 'C:/Users/flori/Documents/Python Scripts/datamining/out1.csv'
path1= 'C:/Users/flori/Documents/Python Scripts/datamining/out2.csv'

In [8]:
col_types = {'repeater': 'enum','exact_item_bought': 'enum'}
col_types1 = {'exact_item_bought': 'enum'}

In [9]:
trainh = h2o.import_file(path=path, col_types=col_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
scoreh = h2o.import_file(path=path1, col_types=col_types1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
trainh['offerdate'] = trainh['offerdate'].as_date("%Y/%m/%d")

In [43]:
scoreh['offerdate'] = scoreh['offerdate'].as_date("%Y/%m/%d")

In [44]:
trainh['month']=trainh['offerdate'].month() 
scoreh['month']=scoreh['offerdate'].month() 

In [46]:
trainh['week']=trainh['offerdate'].week()
scoreh['week']=scoreh['offerdate'].week()

In [47]:
trainh['dayofweek']=trainh['offerdate'].dayOfWeek()
scoreh['dayofweek']=scoreh['offerdate'].dayOfWeek()

In [48]:
trainh['day']=trainh['offerdate'].day()
scoreh['day']=scoreh['offerdate'].day()

In [49]:
trainh['month']=trainh['month'].asfactor()
trainh['week']=trainh['week'].asfactor()

In [50]:
scoreh['month']=scoreh['month'].asfactor()
scoreh['week']=scoreh['week'].asfactor()

In [51]:
training,validation,test = trainh.split_frame([0.4, 0.3],seed=12345)

In [52]:
y = 'repeater'
X = [name for name in training.columns if name not in ['id', 'chain','market','offerdate','quantity', y]]

In [54]:
print(y)
print(X)

repeater
['offer', 'category', 'company', 'offervalue', 'brand', 'avg_purchaseamount', 'avg_purchasequantity', 'exact_item_bought', 'month', 'week', 'dayofweek', 'day']


In [55]:
training[y] = training[y].asfactor()
validation[y] = validation[y].asfactor()
test[y] = test[y].asfactor()

In [56]:
# initialize
repeater_glm = H2OGeneralizedLinearEstimator(family='binomial',
                                         model_id='repeater_glm',
                                         solver='IRLSM',
                                         nfolds=3,
                                         standardize=True,
                                         lambda_search=True,
                                         seed=12345)

# train 
repeater_glm.train(X, y, training_frame=training,validation_frame=validation)

# view detailed results at http://host:ip/flow/index.html

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [57]:
print(repeater_glm.auc(train=True))
print(repeater_glm.model_performance(test_data=validation).auc())

0.6775771253481386
0.6727944090660435


In [58]:
newdf = scoreh[scoreh["id"] == 13584134, :]

In [59]:
repeater_glm.predict(newdf) 

glm prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\flori\Anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'week' has levels not trained on: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
  warnings.warn(w)
C:\Users\flori\Anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'month' has levels not trained on: [5, 6, 7]
  warnings.warn(w)


predict,p0,p1
0,0.895147,0.104853


In [81]:
h2o.cluster().shutdown(prompt=False)

H2O session _sid_b094 closed.
